# Tutorial 15: Hierarchical Agent Teams

In this tutorial, you'll build **nested agent teams** where team supervisors coordinate sub-teams of specialists.

**What you'll learn:**
- Creating team subgraphs with their own supervisors
- Building a top-level supervisor that coordinates teams
- State transformation between hierarchical and team states
- Aggregating results from multiple teams
- When to use hierarchical vs flat multi-agent structures

By the end, you'll have a hierarchical system with a research team and development team, each with their own members and supervisor.

## Prerequisites

- Completed Tutorial 14 (Multi-Agent Collaboration)
- Understanding of the supervisor pattern
- Familiarity with StateGraph and conditional edges

## Why Hierarchical Teams?

The flat supervisor pattern works well for small teams (3-5 agents). But for larger, more complex tasks:

1. **Specialization by domain**: Research team vs Development team vs QA team
2. **Reduced cognitive load**: Each supervisor manages fewer direct reports
3. **Parallel work**: Teams can work independently on their domains
4. **Scalability**: Add new teams without changing the top-level structure

```
                     ┌─────────────────┐
                     │   Top-Level     │
                     │   Supervisor    │
                     └────────┬────────┘
                              │
          ┌───────────────────┴───────────────────┐
          │                                       │
          ▼                                       ▼
   ┌─────────────────┐                     ┌─────────────────┐
   │  Research Team  │                     │  Development    │
   │    Supervisor   │                     │  Team Supervisor│
   └────────┬────────┘                     └────────┬────────┘
            │                                       │
    ┌───────┴───────┐                       ┌───────┴───────┐
    │               │                       │               │
    ▼               ▼                       ▼               ▼
┌─────────┐   ┌─────────┐             ┌─────────┐   ┌─────────┐
│ Searcher│   │ Analyst │             │ Coder   │   │ Tester  │
└─────────┘   └─────────┘             └─────────┘   └─────────┘
```

## Step 1: Setup

In [ ]:
from langgraph_ollama_local import LocalAgentConfig
from langchain_ollama import ChatOllama

config = LocalAgentConfig()
llm = ChatOllama(
    model=config.ollama.model,
    base_url=config.ollama.base_url,
    temperature=0,
)

print(f"Using model: {config.ollama.model}")
print("Setup complete!")

## Step 2: Define Team and Hierarchical States

We need two state types:
1. **TeamState**: For individual teams and their members
2. **HierarchicalState**: For the top-level coordination

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
import operator


class TeamState(TypedDict):
    """State for a single team."""
    messages: Annotated[list, add_messages]
    task: str
    team_name: str
    next_member: str
    member_outputs: Annotated[list[dict], operator.add]
    iteration: int
    max_iterations: int
    team_result: str


class HierarchicalState(TypedDict):
    """State for hierarchical coordination."""
    messages: Annotated[list, add_messages]
    task: str
    active_team: str
    team_results: dict[str, str]  # team_name -> result
    iteration: int
    max_iterations: int
    final_result: str


print("States defined!")
print("\nTeamState: For individual team work")
print("HierarchicalState: For top-level coordination")

## Step 3: Create Team Building Functions

Each team is a complete subgraph with its own supervisor and members.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel, Field
from typing import Literal


def create_team_supervisor(llm, team_name: str, member_names: list[str]):
    """Create a team supervisor node."""
    
    class TeamDecision(BaseModel):
        next_member: str = Field(description="Member name or 'DONE'")
        reasoning: str
    
    structured_llm = llm.with_structured_output(TeamDecision)
    members_list = ", ".join(member_names)
    
    def supervisor(state: TeamState) -> dict:
        outputs = state.get("member_outputs", [])
        progress = "\n".join([
            f"{o['member']}: {o['output'][:200]}..." for o in outputs
        ]) or "No work yet."
        
        decision = structured_llm.invoke([
            SystemMessage(content=f"""You supervise the {team_name} team.
Members: {members_list}
Decide who works next, or say DONE when team task is complete."""),
            HumanMessage(content=f"Task: {state['task']}\nProgress: {progress}")
        ])
        
        return {
            "next_member": decision.next_member,
            "iteration": state["iteration"] + 1,
        }
    
    return supervisor


def create_team_member(llm, member_name: str, role_prompt: str):
    """Create a team member node."""
    
    def member(state: TeamState) -> dict:
        response = llm.invoke([
            SystemMessage(content=role_prompt),
            HumanMessage(content=f"Team task: {state['task']}")
        ])
        
        return {
            "member_outputs": [{
                "member": member_name,
                "output": response.content
            }]
        }
    
    return member


def create_team_finalize(team_name: str):
    """Create team finalization node."""
    
    def finalize(state: TeamState) -> dict:
        outputs = state.get("member_outputs", [])
        result = "\n\n".join([
            f"### {o['member']}\n{o['output']}" for o in outputs
        ])
        return {"team_result": f"## {team_name} Team\n\n{result}"}
    
    return finalize


print("Team building functions defined!")

## Step 4: Build a Team Graph

Let's create a reusable function to build complete team graphs.

In [ ]:
def build_team_graph(llm, team_name: str, members: list[tuple[str, str]]):
    """
    Build a complete team graph.
    
    Args:
        llm: Language model
        team_name: Name of the team
        members: List of (member_name, role_prompt) tuples
    
    Returns:
        Compiled team graph
    """
    member_names = [name for name, _ in members]
    
    workflow = StateGraph(TeamState)
    
    # Add nodes
    workflow.add_node("supervisor", create_team_supervisor(llm, team_name, member_names))
    for name, prompt in members:
        workflow.add_node(name, create_team_member(llm, name, prompt))
    workflow.add_node("finalize", create_team_finalize(team_name))
    
    # Entry
    workflow.add_edge(START, "supervisor")
    
    # Routing
    def route(state):
        if state["iteration"] >= state["max_iterations"]:
            return "finalize"
        if state["next_member"] == "DONE":
            return "finalize"
        if state["next_member"] in member_names:
            return state["next_member"]
        return "finalize"
    
    routing_map = {name: name for name in member_names}
    routing_map["finalize"] = "finalize"
    
    workflow.add_conditional_edges("supervisor", route, routing_map)
    
    # Members return to supervisor
    for name, _ in members:
        workflow.add_edge(name, "supervisor")
    
    workflow.add_edge("finalize", END)
    
    return workflow.compile()


print("Team graph builder defined!")

## Step 5: Create Specific Teams

Now let's create our Research and Development teams.

In [ ]:
# Research Team
research_team = build_team_graph(
    llm,
    "Research",
    members=[
        ("searcher", "You search for information and best practices. Be thorough."),
        ("analyst", "You analyze findings and identify key insights. Be concise."),
    ]
)

# Development Team
dev_team = build_team_graph(
    llm,
    "Development",
    members=[
        ("coder", "You write clean, well-documented code. Follow best practices."),
        ("tester", "You write tests and identify edge cases. Be thorough."),
    ]
)

print("Teams created!")
print("- Research Team: searcher, analyst")
print("- Development Team: coder, tester")

## Step 6: Test a Team Independently

Each team can work independently before we connect them hierarchically.

In [ ]:
# Test the research team
research_result = research_team.invoke({
    "messages": [],
    "task": "Research best practices for input validation in Python",
    "team_name": "Research",
    "next_member": "",
    "member_outputs": [],
    "iteration": 0,
    "max_iterations": 4,
    "team_result": "",
})

print("Research Team Result:")
print("="*50)
print(research_result["team_result"])

## Step 7: Create the Top-Level Supervisor

The top supervisor coordinates between teams.

In [ ]:
def create_top_supervisor(llm, team_names: list[str]):
    """Create top-level supervisor."""
    
    class TopDecision(BaseModel):
        next_team: str = Field(description="Team name or 'FINISH'")
        reasoning: str
    
    structured_llm = llm.with_structured_output(TopDecision)
    teams_list = ", ".join(team_names)
    
    def supervisor(state: HierarchicalState) -> dict:
        results = state.get("team_results", {})
        progress = "\n".join([
            f"{team}: {result[:200]}..." for team, result in results.items()
        ]) or "No teams have reported yet."
        
        decision = structured_llm.invoke([
            SystemMessage(content=f"""You coordinate teams: {teams_list}
Assign work to the right team. Say FINISH when overall task is complete.
Typically: research first, then development."""),
            HumanMessage(content=f"Task: {state['task']}\nTeam progress: {progress}")
        ])
        
        return {
            "active_team": decision.next_team,
            "iteration": state["iteration"] + 1,
        }
    
    return supervisor


print("Top supervisor creator defined!")

## Step 8: Create Team Wrappers

We need to wrap team graphs as nodes that transform state correctly.

In [ ]:
def create_team_wrapper(team_graph, team_name: str):
    """Wrap a team graph as a node in the hierarchical graph."""
    
    def team_node(state: HierarchicalState) -> dict:
        # Transform to team state
        team_input = {
            "messages": [],
            "task": state["task"],
            "team_name": team_name,
            "next_member": "",
            "member_outputs": [],
            "iteration": 0,
            "max_iterations": 4,
            "team_result": "",
        }
        
        # Run team
        team_output = team_graph.invoke(team_input)
        
        # Update hierarchical state
        new_results = state.get("team_results", {}).copy()
        new_results[team_name] = team_output.get("team_result", "")
        
        return {"team_results": new_results}
    
    return team_node


def create_aggregate():
    """Create aggregation node."""
    
    def aggregate(state: HierarchicalState) -> dict:
        results = state.get("team_results", {})
        final = "\n\n---\n\n".join([
            f"# {team}\n\n{result}" for team, result in results.items()
        ])
        return {"final_result": final}
    
    return aggregate


print("Team wrapper and aggregation defined!")

## Step 9: Build the Hierarchical Graph

Now we connect everything together.

In [ ]:
# Define our teams
teams = {
    "research": research_team,
    "development": dev_team,
}

team_names = list(teams.keys())

# Build hierarchical graph
hierarchical = StateGraph(HierarchicalState)

# Add top supervisor
hierarchical.add_node("top_supervisor", create_top_supervisor(llm, team_names))

# Add team nodes
for name, graph in teams.items():
    hierarchical.add_node(name, create_team_wrapper(graph, name))

# Add aggregate
hierarchical.add_node("aggregate", create_aggregate())

# Entry
hierarchical.add_edge(START, "top_supervisor")

# Routing
def route_top(state):
    if state["iteration"] >= state["max_iterations"]:
        return "aggregate"
    if state["active_team"] == "FINISH":
        return "aggregate"
    if state["active_team"].lower() in team_names:
        return state["active_team"].lower()
    return "aggregate"

routing_map = {name: name for name in team_names}
routing_map["aggregate"] = "aggregate"

hierarchical.add_conditional_edges("top_supervisor", route_top, routing_map)

# Teams return to top supervisor
for name in team_names:
    hierarchical.add_edge(name, "top_supervisor")

hierarchical.add_edge("aggregate", END)

# Compile
hierarchical_graph = hierarchical.compile()

print("Hierarchical graph compiled!")
print("\nStructure:")
print("  top_supervisor -> [research | development | aggregate]")
print("  research (team) -> top_supervisor")
print("  development (team) -> top_supervisor")
print("  aggregate -> END")

## Step 10: Run the Hierarchical System

In [ ]:
def run_hierarchical_task(task: str, max_iterations: int = 6):
    """Run a task through the hierarchical system."""
    
    print("="*60)
    print(f"Task: {task}")
    print("="*60)
    
    result = hierarchical_graph.invoke({
        "messages": [],
        "task": task,
        "active_team": "",
        "team_results": {},
        "iteration": 0,
        "max_iterations": max_iterations,
        "final_result": "",
    })
    
    print("\n" + "="*60)
    print("FINAL RESULT")
    print("="*60)
    print(result["final_result"])
    
    return result


# Test with a full task
result = run_hierarchical_task(
    "Create a Python function to validate and sanitize user email input"
)

## Step 11: Using the Built-in Module

The `langgraph_ollama_local.agents.hierarchical` module provides these patterns ready to use:

In [ ]:
from langgraph_ollama_local.agents.hierarchical import (
    create_team_graph,
    create_hierarchical_graph,
    run_hierarchical_task as run_task,
)

# Create teams using the module
research = create_team_graph(
    llm, "research",
    members=[
        ("searcher", "Search for information and solutions.", None),
        ("analyst", "Analyze findings and provide insights.", None),
    ]
)

development = create_team_graph(
    llm, "development",
    members=[
        ("coder", "Write clean, documented code.", None),
        ("tester", "Write tests and check edge cases.", None),
    ]
)

# Create hierarchical graph
graph = create_hierarchical_graph(
    llm,
    {"research": research, "development": development}
)

# Run a task
result = run_task(graph, "Build a password strength checker", max_iterations=5)
print(result["final_result"][:1000])

## Complete Code

In [ ]:
# === Complete Hierarchical Teams Implementation ===

from typing import Annotated
from typing_extensions import TypedDict
import operator

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field

from langgraph_ollama_local import LocalAgentConfig


# === States ===
class TeamState(TypedDict):
    messages: Annotated[list, add_messages]
    task: str
    team_name: str
    next_member: str
    member_outputs: Annotated[list[dict], operator.add]
    iteration: int
    max_iterations: int
    team_result: str

class HierarchicalState(TypedDict):
    messages: Annotated[list, add_messages]
    task: str
    active_team: str
    team_results: dict[str, str]
    iteration: int
    max_iterations: int
    final_result: str


# === Team Builder ===
def build_team(llm, name: str, members: list[tuple[str, str]]):
    member_names = [n for n, _ in members]
    
    class Decision(BaseModel):
        next_member: str
        reasoning: str
    
    def supervisor(state):
        decision = llm.with_structured_output(Decision).invoke([
            SystemMessage(content=f"Supervise {name} team: {member_names}. Say DONE when finished."),
            HumanMessage(content=f"Task: {state['task']}")
        ])
        return {"next_member": decision.next_member, "iteration": state["iteration"] + 1}
    
    def make_member(n, prompt):
        def member(state):
            r = llm.invoke([SystemMessage(content=prompt), HumanMessage(content=state["task"])])
            return {"member_outputs": [{"member": n, "output": r.content}]}
        return member
    
    def finalize(state):
        return {"team_result": "\n".join([o["output"] for o in state.get("member_outputs", [])])}
    
    def route(state):
        if state["iteration"] >= state["max_iterations"] or state["next_member"] == "DONE":
            return "finalize"
        return state["next_member"] if state["next_member"] in member_names else "finalize"
    
    g = StateGraph(TeamState)
    g.add_node("supervisor", supervisor)
    for n, p in members:
        g.add_node(n, make_member(n, p))
    g.add_node("finalize", finalize)
    g.add_edge(START, "supervisor")
    g.add_conditional_edges("supervisor", route, {n: n for n in member_names} | {"finalize": "finalize"})
    for n, _ in members:
        g.add_edge(n, "supervisor")
    g.add_edge("finalize", END)
    return g.compile()


# === Hierarchical Builder ===
def build_hierarchical(llm, teams: dict):
    names = list(teams.keys())
    
    class Decision(BaseModel):
        next_team: str
        reasoning: str
    
    def top_supervisor(state):
        decision = llm.with_structured_output(Decision).invoke([
            SystemMessage(content=f"Coordinate teams: {names}. Say FINISH when done."),
            HumanMessage(content=f"Task: {state['task']}")
        ])
        return {"active_team": decision.next_team, "iteration": state["iteration"] + 1}
    
    def wrap_team(graph, name):
        def node(state):
            out = graph.invoke({"messages": [], "task": state["task"], "team_name": name,
                "next_member": "", "member_outputs": [], "iteration": 0, "max_iterations": 4, "team_result": ""})
            new = state.get("team_results", {}).copy()
            new[name] = out.get("team_result", "")
            return {"team_results": new}
        return node
    
    def aggregate(state):
        return {"final_result": "\n\n".join([f"# {k}\n{v}" for k, v in state.get("team_results", {}).items()])}
    
    def route(state):
        if state["iteration"] >= state["max_iterations"] or state["active_team"] == "FINISH":
            return "aggregate"
        return state["active_team"].lower() if state["active_team"].lower() in names else "aggregate"
    
    g = StateGraph(HierarchicalState)
    g.add_node("top_supervisor", top_supervisor)
    for name, graph in teams.items():
        g.add_node(name, wrap_team(graph, name))
    g.add_node("aggregate", aggregate)
    g.add_edge(START, "top_supervisor")
    g.add_conditional_edges("top_supervisor", route, {n: n for n in names} | {"aggregate": "aggregate"})
    for name in names:
        g.add_edge(name, "top_supervisor")
    g.add_edge("aggregate", END)
    return g.compile()


# === Use ===
if __name__ == "__main__":
    cfg = LocalAgentConfig()
    llm = ChatOllama(model=cfg.ollama.model, base_url=cfg.ollama.base_url, temperature=0)
    
    teams = {
        "research": build_team(llm, "research", [("searcher", "Search info"), ("analyst", "Analyze")]),
        "dev": build_team(llm, "dev", [("coder", "Write code"), ("tester", "Test code")]),
    }
    
    graph = build_hierarchical(llm, teams)
    result = graph.invoke({"messages": [], "task": "Build a URL validator",
        "active_team": "", "team_results": {}, "iteration": 0, "max_iterations": 5, "final_result": ""})
    print(result["final_result"])

## Key Concepts

| Concept | Description |
|---------|-------------|
| **Team Subgraphs** | Each team is a complete graph with supervisor and members |
| **Top Supervisor** | Coordinates between teams at the highest level |
| **State Transformation** | Convert between hierarchical and team states |
| **Team Wrappers** | Wrap team graphs as nodes in parent graph |
| **Result Aggregation** | Combine all team outputs at the end |
| **Nested Iteration** | Each level has its own iteration limits |

## When to Use Hierarchical vs Flat

| Use Hierarchical When | Use Flat When |
|----------------------|---------------|
| Many agents (6+) | Few agents (3-5) |
| Clear domain boundaries | Agents collaborate closely |
| Teams can work independently | Work is highly sequential |
| Need organizational structure | Simple task decomposition |

## What's Next

- [Tutorial 16: Subgraph Patterns](16_subgraphs.ipynb) - Composable, reusable graph components